In [1]:
%matplotlib inline
import theano
import pymc3 as pm
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
sns.set_style('white')
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import theano.tensor as tt
from theano import shared
import pandas as pd
import scipy as sp

In [2]:
df = pd.read_csv('./pb/pb_asc_x1x5_x6.csv', header=None, names=['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
df.head()

,x1,x2,x3,x4,x5,x6
0,16,30,41,48,53,16
1,1,26,41,50,57,11
2,23,33,42,46,59,4
3,2,18,19,22,63,19
4,10,16,40,52,55,17


In [3]:
x1_shift = shared(df['x1'].values[:, np.newaxis], broadcastable=(False, True))
#x1_shift = df['x1'].values[:, np.newaxis]
x2obs = df['x2'].values[:, np.newaxis]
K = 30
with pm.Model() as model:
    alpha = pm.Gamma('alpha', 1., 1., shape=K)
    beta = pm.Gamma('beta', 1., 1., shape=K)
    
    gamma1 = pm.Uniform('gamma1', lower=1, upper=3, shape=K)
    gamma2 = pm.Uniform('gamma2', lower=-1, upper=1, shape=K)
    gamma3 = pm.Uniform('gamma3', lower=-1, upper=1, shape=K)
    
    shmu = pm.Deterministic('shmu', gamma1*x1_shift ) #+ gamma2*alpha*np.sqrt(x1_shift) + gamma3*beta*np.log(x1_shift))

    delta = pm.Gamma('delta', 1., 1., shape=K)
    
    #mixture components
    components = pm.NegativeBinomial.dist(mu=shmu, alpha=delta, shape=K)
    
    #mixture component weights
    w = pm.Dirichlet('w', a=np.array([1]*K)) 
    
    like = pm.Mixture('like', w=w, comp_dists=components, observed=x2obs)


In [4]:
theano.config.exception_verbosity='high'
theano.config.mode='DebugMode'
with model:
    x2trace = pm.sample(100, n_init=1000)
    pm.sample_ppc(x2trace, 10)
    x2ppc = pm.traceplot(x2trace, varnames=['w', 'shmu'])

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
  0%|          | 0/1000 [00:00<?, ?it/s]

InvalidValueError: InvalidValueError
        type(variable) = TensorType(float32, scalar)
        variable       = TensorConstant{-inf}
        type(value)    = <class 'numpy.ndarray'>
        dtype(value)   = float32
        shape(value)   = ()
        value          = -inf
        min(value)     = -inf
        max(value)     = -inf
        isfinite       = False
        client_node    = None
        hint           = Graph Input 'TensorConstant{-inf}' has invalid value -inf
        specific_hint  = none
        context        = ...
  TensorConstant{-inf} [id A]

        

In [13]:
import sys
print(sys.version)
print(sys.version_info)
print(theano.version)
print('pymc3: ', pm.__version__)

3.6.0 |Anaconda custom (x86_64)| (default, Dec 23 2016, 13:19:00) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)
<module 'theano.version' from '/Users/vmullachery/anaconda/envs/dl3.6/lib/python3.6/site-packages/theano/version.py'>
pymc3:  3.0
